In [32]:
%matplotlib inline
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400

# Проверка гипотезы распределения используя критерий Пирсона
. Сгенерировать выборку объёмом 100 элементов для нормального распределения $N(x, 0, 1)$. По сгенерированной выборке оценить параметры 𝜇 и 𝜎 нормального закона методом максимального правдоподобия.
В качестве основной гипотезы 𝐻0 будем считать, что сгенерированное
распределение имеет вид $N(x, \hat{\mu}, \hat{\sigma})$. Проверить основную гипотезу, используя критерий согласия $\chi^2 $
. В качестве уровня значимости взять
𝛼 = 0.05. Привести таблицу вычислений $\chi^2 $
.
Исследовать точность (чувствительность) критерия $\chi^2 $
- сгенерировать выборки равномерного распределения и распределения Лапласа
малого объема (например, 20 элементов). Проверить их на нормальность.


In [33]:
# размер выборки
N = 100

# выборка:
data = st.norm.rvs(size=N)



Метод максимального правдоподобия (о.м.п.)

In [34]:
def OMP(array):
    mean = array.mean()
    std = array.std()
    return {
        'mean' : mean,
        'std' : std
    }

Критерий Пирсона для оценки гипотезы о распределении

In [55]:
def Pirson(data,func):
    N = len(data)
    
    # extimation of sigma and mean
    L = OMP(data)
    func = func(L['mean'],L['std'])
    print("mean : ",L['mean'])
    print("std : ",L['std'])
    
    


    # степени свободы
    CONST_DEGREES_OF_FREEDOM = 10

    # начальное значение и конечное значение отрезка
    start = min(data)
    end = max(data)
    val = np.linspace(start,end,CONST_DEGREES_OF_FREEDOM - 1)
    
    probablity = []
    probablity_sel = []
    for i in range(0,len(val) + 1):
        if i == 0:
            probablity.append(func.cdf(val[i]))
            probablity_sel.append(sum(data <= val[i]))
        elif i == len(val):
            probablity.append(1 - func.cdf(val[i - 1]))
            probablity_sel.append(sum(data > val[i - 1]))
        else:
            probablity.append(func.cdf(val[i]) -func.cdf(val[i - 1]) )
            probablity_sel.append(sum(np.bitwise_and(data <= val[i],data > val[i - 1])))
       
    ni__npi = 0
    npi = 0
    pi = 0
    hi = 0
    n = 0
    print("n\tpi\tnpi\tni - npi\t(ni - npi)/(npi)")
    for i in range(CONST_DEGREES_OF_FREEDOM):
        hi += (probablity_sel[i] - N*probablity[i])**2/(N * probablity[i] + 0.001)
        n += probablity_sel[i]
        pi += probablity[i]
        npi += N*probablity[i]
        ni__npi += probablity[i] - N*probablity[i]
        print(str(probablity_sel[i]),
            format(probablity[i],'.2f'),
            format(N*probablity[i],'.2f'),
            format(probablity[i] - N*probablity[i],'.2f') + "\t",
            format((probablity_sel[i] - N*probablity[i])**2/(N * probablity[i]  +0.001),'.2f') ,
             sep="\t")
    print(str(n),
        format(pi,'.2f'),
        format(npi,'.2f'),
        format(ni__npi,'.2f') + "\t",
        format(hi,'.2f') ,
         sep="\t")
    print(val)
    return hi,CONST_DEGREES_OF_FREEDOM



In [56]:
Pirson(data,st.norm)

mean :  0.059999454477974494
std :  0.9390321292045365
n	pi	npi	ni - npi	(ni - npi)/(npi)
1	0.01	0.54	-0.54		0.38
2	0.02	2.45	-2.42		0.08
9	0.08	8.17	-8.09		0.08
20	0.18	17.82	-17.65		0.27
20	0.25	25.41	-25.15		1.15
22	0.24	23.67	-23.44		0.12
19	0.14	14.42	-14.28		1.45
5	0.06	5.74	-5.68		0.10
2	0.01	1.49	-1.48		0.17
0	0.00	0.28	-0.28		0.28
100	1.00	100.00	-99.00		4.08
[-2.33081573 -1.70720796 -1.08360019 -0.45999243  0.16361534  0.7872231
  1.41083087  2.03443864  2.6580464 ]


(4.0849580608467795, 10)

In [57]:
Pirson(data,st.laplace)

mean :  0.059999454477974494
std :  0.9390321292045365
n	pi	npi	ni - npi	(ni - npi)/(npi)
1	0.04	3.92	-3.88		2.17
2	0.04	3.70	-3.66		0.78
9	0.07	7.18	-7.11		0.46
20	0.14	13.95	-13.81		2.63
20	0.26	26.48	-26.22		1.59
22	0.22	21.73	-21.51		0.00
19	0.11	11.18	-11.07		5.46
5	0.06	5.76	-5.70		0.10
2	0.03	2.96	-2.93		0.31
0	0.03	3.14	-3.11		3.14
100	1.00	100.00	-99.00		16.65
[-2.33081573 -1.70720796 -1.08360019 -0.45999243  0.16361534  0.7872231
  1.41083087  2.03443864  2.6580464 ]


(16.64830292214529, 10)

In [58]:
Pirson(data,st.uniform)

mean :  0.059999454477974494
std :  0.9390321292045365
n	pi	npi	ni - npi	(ni - npi)/(npi)
1	0.00	0.00	0.00		1000.00
2	0.00	0.00	0.00		4000.00
9	0.00	0.00	0.00		81000.00
20	0.00	0.00	0.00		400000.00
20	0.11	11.03	-10.92		7.28
22	0.66	66.41	-65.75		29.70
19	0.23	22.56	-22.33		0.56
5	0.00	0.00	0.00		25000.00
2	0.00	0.00	0.00		4000.00
0	0.00	0.00	0.00		0.00
100	1.00	100.00	-99.00		515037.54
[-2.33081573 -1.70720796 -1.08360019 -0.45999243  0.16361534  0.7872231
  1.41083087  2.03443864  2.6580464 ]


(515037.5420572824, 10)